In [24]:
# Predictions for ensembling
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import numpy as np
from sklearn.model_selection import train_test_split
import torch

CHECKPOINT = "checkpoint-1000"
BERT = "bert-base-uncased"

bert_tokenizer = AutoTokenizer.from_pretrained(BERT)
bert_model = AutoModelForMultipleChoice.from_pretrained(f"./results/{BERT}/{CHECKPOINT}/")

options = ["OptionA", "OptionB", "OptionC"]
answers = ['A', 'B', 'C']


In [39]:
def ensemble(example):
    prompts = []
    for opt in options:
        prompts.append([example['FalseSent'], example[opt]])
        
#     print(prompts)

    bert_inputs = bert_tokenizer(prompts, return_tensors="pt", padding=True)

    labels = torch.tensor(0).unsqueeze(0)
    
    bert_outputs = bert_model(**{k: v.unsqueeze(0) for k, v in bert_inputs.items()}, labels=labels)

    predicted_class = bert_outputs.logits.argmax().item()

    return answers[predicted_class]


In [40]:

train_file = "common-sense/train_data.csv"
df_train = pd.read_csv(train_file)
ans_file = "common-sense/train_answers.csv"
df_ans = pd.read_csv(ans_file)
test_file = "common-sense/test_data.csv"
df_test = pd.read_csv(test_file)
X_train, X_test, y_train, y_test = train_test_split(df_train, df_ans['answer'], test_size=0.3, random_state=40)
X_train.dtypes

    # submission_df = df_test[["id", "label"]].copy()
    # submission_df = submission_df.rename(columns={"label": "answer"})
    # submission_df.set_index("id", inplace=True)
    # submission_df.to_csv(f"submissions/Submission_12-12-2022_1.csv")

id           object
FalseSent    object
OptionA      object
OptionB      object
OptionC      object
dtype: object

In [41]:
df_test.dtypes

id           object
FalseSent    object
OptionA      object
OptionB      object
OptionC      object
dtype: object

In [42]:
df_dr = X_test.apply(lambda row: ensemble(row), axis=1)

print(df_dr)

5324    A
7392    A
6146    B
2242    C
4100    C
       ..
1912    B
2738    B
3606    C
1148    A
5197    B
Length: 2400, dtype: object


In [50]:
df_predictions = df_dr.to_frame()
df_predictions.columns = ['predicted']

,predicted
5324,A
7392,A
6146,B
2242,C
4100,C
...,...
1912,B
2738,B
3606,C
1148,A


In [57]:
df_predictions['actual'] = y_test
# df_predictions.set_index(0, inplace=True)
df_predictions

,predicted,actual
5324,A,A
7392,A,A
6146,B,B
2242,C,C
4100,C,C
...,...,...
1912,B,B
2738,B,B
3606,C,C
1148,A,A


In [58]:
df_predictions.to_csv(f"submissions/30_percent_trainset.csv")